In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# preprocessing library
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer

In [ ]:
# data visualization library
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
# deep learning library
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, BatchNormalization
from tensorflow.keras.layers import Dense, Flatten

In [ ]:
# read train data
train_dataset = pd.read_csv("/kaggle/input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv")

# read test data
test_dataset = pd.read_csv("/kaggle/input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv")

In [ ]:
# sample train data
train_dataset.head(10)

In [ ]:
# check the size of the trianing data
print("There are {} rows and {} columns".format(train_dataset.shape[0], train_dataset.shape[1]))


In [ ]:
# sample test data
test_dataset.head(10)

In [ ]:
# check the size of test data
print("There are {} rows and {} columns".format(test_dataset.shape[0], test_dataset.shape[1]))

In [ ]:
# check the amount of data present in each of the labels in training dataset
# we will do this through data visualization
plt.figure(figsize=(7,7))
sns.countplot(train_dataset['label'], palette='Set3')
plt.show()

The data is uniformly distributed

In [ ]:
# define the independent variable
y_train = train_dataset['label']
y_test = test_dataset['label']

In [ ]:
# delete the independent variable from both dataset
# dataset contains the pixel values only
del train_dataset['label']
del test_dataset['label']

In [ ]:
# define the dependent variable and reshape it
# Using channel 1 to transform images into greyscale
# There are 784 pixel columns which makes it 28x28(=784)
x_train = train_dataset.values.reshape(-1, 28, 28, 1)
x_test = test_dataset.values.reshape(-1, 28, 28, 1)

print("Shape of x train data- {}".format(x_train.shape))
print("Shape of x test data- {}".format(x_test.shape))

In [ ]:
# divide with 255 to scale values down to range of 0...1
x_train_preprocessed = x_train / 255
x_test_preprocessed = x_test / 255

In [ ]:
# tranforming multi class data to 0/1 array data (similar to one hot encoding)
label_binarizer_obj = LabelBinarizer()
y_train_preprocessed = label_binarizer_obj.fit_transform(y_train)
y_test_preprocessed = label_binarizer_obj.fit_transform(y_test)
print("Shape of y train data- {}".format(y_train_preprocessed.shape))
print("Shape of y test data- {}".format(y_test_preprocessed.shape))

In [ ]:
# sample y data
y_train_preprocessed[0]

Data is ready!!

Variables are-

x_train_preprocessed

x_test_preprocessed

y_train_preprocessed

y_test_preprocessed

In [ ]:
# sample image
plt.imshow(x_train_preprocessed[1])
plt.title("Class- {}".format(y_train_preprocessed[1].argmax()))
plt.show()

In [ ]:
# some random images
import random
plt.figure(figsize=(12,12))
n = 0
for i in range(16):
    n += 1
    plt.subplot(4, 4, n)
    plt.subplots_adjust(hspace = 0.5)
    index = random.randint(0, y_train_preprocessed.shape[0])
    plt.imshow(x_train_preprocessed[index])
    plt.title("Class- {}".format(y_train_preprocessed[index].argmax()))
plt.show()
    


In [ ]:
# Data augmentation
# Generate new image data to increase the training data size
# Alter the images by horizontally flipping it, shifting its height and width, zooming and rotating it
datagen_object = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range = 10,
                                                                height_shift_range = 0.2,
                                                                width_shift_range = 0.2,
                                                                zoom_range = 0.2,
                                                                horizontal_flip = True)


Create model, fit data and check accuracy. If accuracy is less, increase the size of the dataset by duplicating it using image data generator.

In [ ]:
# model creation
sign_language_MNIST_model = tf.keras.models.Sequential()

sign_language_MNIST_model.add(Conv2D(128, (3, 3), activation = 'relu', input_shape = (28, 28, 1)))
sign_language_MNIST_model.add(BatchNormalization())
sign_language_MNIST_model.add(MaxPool2D(2, 2))


sign_language_MNIST_model.add(Conv2D(64, (3, 3), activation = 'relu'))
sign_language_MNIST_model.add(BatchNormalization())
sign_language_MNIST_model.add(MaxPool2D(2, 2))

sign_language_MNIST_model.add(Conv2D(32, (3, 3), activation = 'relu'))
sign_language_MNIST_model.add(BatchNormalization())
sign_language_MNIST_model.add(MaxPool2D(2, 2))

sign_language_MNIST_model.add(Flatten())

sign_language_MNIST_model.add(Dense(512, activation = 'relu'))
sign_language_MNIST_model.add(Dropout(0.2))
sign_language_MNIST_model.add(Dense(24, activation = 'softmax'))  # 24 number of output classes


In [ ]:
# print the summary of the model, check trainable parameters
sign_language_MNIST_model.summary()

In [ ]:
# compile model
sign_language_MNIST_model.compile(optimizer = 'adam',
                                 loss = 'categorical_crossentropy',
                                 metrics = ['accuracy'])

In [ ]:
# fit the train data
sign_language_MNIST_model.fit(datagen_object.flow(x_train_preprocessed, y_train_preprocessed, batch_size = 128), epochs = 20)

In [ ]:
# accuracy on the test set
loss, accuracy = sign_language_MNIST_model.evaluate(x_test_preprocessed, y_test_preprocessed)
print("Accuracy on the test set- {}%".format(accuracy*100))

In [ ]:
# trail 1 -> Accuracy = 86.58% || Highly Overfitting 
# Increase the nodes
# trail 2 -> Accuracy = 89.80% || Still Overfitting
# Data augmentation with BatchNormalization
# trail 3 -> Accuracy = 96.96% || More than the training accuracy
# Augmented Data without BacthNormalization
# trail 4 -> Accuracy = 86.53% || Highly Underfitting
# Augmented Data with BatchNormalization layer after activation layer
# trail 5 -> Accuracy = 90.27% || 